In [15]:
import numpy as np
import pandas as pd
from unidecode import unidecode
import re

In [16]:
df = pd.read_csv('clean/divar_out_remove.csv')

In [17]:
def ConvertNumbers(persian_int):
        return unidecode(f"{persian_int}")

df['floor_en'] = df['floor'].apply(ConvertNumbers)
df.drop(df[df['floor_en']=='nan'].index, inplace=True)


def tabaghe_t(x):
    result=None
    floor=x.split()
    if 'hmkhf' in x:
        result=('0',floor[-1])
    elif 'zyrhmkhf' in x:
        result=('-1',floor[-1])
    elif 'byshtr' in x:
        result=(floor[0],floor[-1])
    else:
        result= re.findall ('\d+',x)
    return result

def total_floor(x):
    try:
        return x[1]
    except:
        return x[0]

floors=df.floor_en.apply(tabaghe_t)
df['floor_en']=floors.apply(lambda x: x[0])
df['total_floor']=floors.apply(total_floor)
df.total_floor.replace(to_replace='hmkhf', value='0', inplace=True)
df.total_floor.replace(to_replace='zyrhmkhf', value='-1', inplace=True)
df.drop(['floor'], axis =1, inplace =True)

In [18]:
df.loc[:,'elevator']=df['elevator'].fillna(0)
df.loc[:,'parking']=df['parking'].fillna(0)
df.loc[:,'storage']=df['storage'].fillna(0)

In [19]:
df.drop(['lat','long','Unnamed: 0.1','Unnamed: 0','slug','category_title','latin_city','divder','total_cost','city','token'], axis =1, inplace =True)

In [20]:
df.drop(df[df['floor_en']=='30+'].index,axis =0,inplace=True)

df.loc[:,'room'] = df.loc[:,'room'].astype(str).str.replace("بدون اتاق", "0")
df['room'] = df['room'].apply(ConvertNumbers)
df.loc[:,'room'] = pd.to_numeric(df.loc[:,'room'].astype(str).str.replace("4+", "5"))
df['built_year'] = df['built_year'].apply(ConvertNumbers)
df.loc[:,'built_year'] = pd.to_numeric(df.loc[:,'built_year'].astype(str).str.replace("qbl z", ""))


/tmp/ipykernel_48738/431198150.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:,'room'] = pd.to_numeric(df.loc[:,'room'].astype(str).str.replace("4+", "5"))


In [22]:
num = [1,8,9]
for i in num:
    df.loc[:,df.columns[i]] = pd.to_numeric(df[df.columns[i]].astype(str).str.replace("٬", "").apply(unidecode), errors='coerce')
   

In [26]:
df.loc[:,'room']=df['room'].fillna('5')
df.dropna(subset=['district'],inplace = True, axis=0)
df.dropna(subset=['room'],inplace = True, axis=0)

In [28]:
# ####  convert distric to numeric value
district_gp_by = df.groupby("district").price_per_meter.median().sort_values()
values = district_gp_by.index
dist_weight = range(1, len(values)+1)
dist_dic = {val: weights for (val, weights) in zip(values, dist_weight)}
df.district.replace(dist_dic, inplace=True)

In [29]:
df.to_csv("clean/preprocess_csv.csv")